In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import replicaEVSE.datautils as simdu
import os
import joblib
import dask.dataframe as dd
from tqdm import tqdm

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../data/'
mode = 'PRIVATE_AUTO'
test = False

# Sample the number of people/vehicles by county segment and year

see  `scripts/sample_counties.py` to run 

In [8]:
np.arange(2034, 2036, 1)

array([2034, 2035])

In [6]:


#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv(datadir+'EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))
    df = df.head(10000)
    df = df.loc[df['mode'] == mode]
    simulation_id = 'dev'

else: 
    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]
    # take out the mobile and commercial MHDV

### TODO: revisit taking out mobile home owners
df = df[(df['building_type'] != 'mobile') & (df['building_type'] != None)]



###  sample by county: for now use a percent until we have the stock rollover model

`ev_df` should be given to us to run. 

In [3]:
stock_rollover = pd.read_csv(datadir+'ldv_population_output_adjusted.csv')
efficiency = pd.read_csv(datadir+'vehicle_inputs.csv')

In [4]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']
for cartype in personal + commercial:
    for powertrain in  ['EV', 'PHEV']:
        cond = (efficiency['Vehicle_type']==cartype) & ~efficiency['Powertrain'].isin(['ICE-G', 'ICE-D', 'FCEV']) & (efficiency['Powertrain']==powertrain) & (efficiency['Vintage'] == 2022)
        ef = efficiency[cond]['Efficiency'].values[0]
        print(cartype, powertrain, ef)

Personal Sedan EV 0.25
Personal Sedan PHEV 80.0
Personal Crossover EV 0.3
Personal Crossover PHEV 80.0
Personal Truck/SUV EV 0.49
Personal Truck/SUV PHEV 80.0
Commercial Sedan EV 0.25
Commercial Sedan PHEV 80.0
Commercial Crossover EV 0.3
Commercial Crossover PHEV 80.0
Commercial Truck/SUV EV 0.49
Commercial Truck/SUV PHEV 80.0


# extract the population ldv data from the stock rollover model 
 - What to do about housing == 'other'? 

### include both EV and PHEV

In [5]:
ev_cond = stock_rollover['Powertrain'].isin(['EV', 'PHEV']).reset_index(drop=True)
# ev_cond = (stock_rollover['Powertrain']=='EV') | (stock_rollover['Powertrain']=='PHEV')
# nev_df = stock_rollover[(stock_rollover['Powertrain']=='EV')].copy()
nev_df = stock_rollover[ev_cond].copy()
nev_df = nev_df[nev_df['domicile'] != 'other'].copy()
nev_df.drop(columns=['Unnamed: 0'], inplace=True)
print(len(nev_df))


924


See the change in stock over the years

In [13]:
nev_df[['2022', '2035']]

,2022,2035
0,0,1
3,0,0
4,0,5
7,0,1
8,0,0
...,...,...
1839,98,14859
1840,34,22314
1843,18,5020
1844,133,83107


We ingest the number of vehicles in each segment for each year and sample from that subset of the trips table for each county segment and year. This takes > 300 minutes for all the years and segments

In [14]:
nev_df

,County,Vehicle_type,domicile,Powertrain,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,Adams,Commercial Crossover,mfh,EV,0,0,0,0,0,0,0,1,1,1,1,1,1,1
3,Adams,Commercial Crossover,mfh,PHEV,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Adams,Commercial Crossover,sfh,EV,0,0,1,1,1,2,2,2,3,3,3,5,5,5
7,Adams,Commercial Crossover,sfh,PHEV,0,1,1,1,1,1,1,1,1,1,1,1,1,1
8,Adams,Commercial Sedan,mfh,EV,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,Yakima,Personal Sedan,sfh,PHEV,98,2433,4815,6453,7994,9117,10046,10703,11606,12252,12842,13552,14206,14859
1840,Yakima,Personal Truck/SUV,mfh,EV,34,1780,3504,5143,6768,8393,9963,11285,12851,14524,16279,18064,19987,22314
1843,Yakima,Personal Truck/SUV,mfh,PHEV,18,1063,1757,2265,2617,2945,3170,3367,3593,3845,4102,4380,4667,5020
1844,Yakima,Personal Truck/SUV,sfh,EV,133,6521,12813,18796,24713,30637,36335,41141,46932,53211,59811,66703,74132,83107


In [15]:
# get the unique people in the dataframe
pop_df = df.drop_duplicates(subset=['person_id'])[['person_id', 'destination_county', 'building_type']]

In [13]:
# test run on a single year ~14 mins
final_df = simdu.sample_people_by_county(df, nev_df, year=2025, fraction=None)

Selecting people from each county in year=2025...14 mins per year


In [14]:
final_df.head()[['person_id', 'distance_miles', 'destination_county', 'building_type', 'engine', 'segment', 'efficiency', 'year']]

,person_id,distance_miles,destination_county,building_type,engine,segment,efficiency,year
23982863,3105704646168074816,4.782405,"Adams County, WA",single_family,EV,Commercial Crossover,0.3,2025
23982866,3105704646168074816,0.176954,"Adams County, WA",single_family,EV,Commercial Crossover,0.3,2025
23982867,3105704646168074816,26.754191,"Grant County, WA",single_family,EV,Commercial Crossover,0.3,2025
23982868,3105704646168074816,32.847668,"Adams County, WA",single_family,EV,Commercial Crossover,0.3,2025
23982863,3105704646168074816,2.630323,"Adams County, WA",single_family,PHEV,Commercial Crossover,0.9,2025


### Can parallelize this by looping over years

In [16]:
year_list = np.arange(2022, 2036, 1)
joblib.Parallel(verbose=10, n_jobs=4)(joblib.delayed(simdu.run_and_save_sampled_populations)(
    df,
    nev_df, 
    year,
    datadir,
    ) for year in year_list)


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Selecting people from each county in year=2022...14 mins per year
Selecting people from each county in year=2023...14 mins per year
Selecting people from each county in year=2024...14 mins per year
Selecting people from each county in year=2025...14 mins per year


Bad pipe message: %s [b'\x06s\xaa\x98k\xcf\xb3s\x8c9\xaf\xe0\xa9\x91\xb7K@A @Ky\x99S\xd3\x88\x97UR\x9e\xcaT\\/Sz\xb1\xd6B\x15']
Bad pipe message: %s [b'\xbc\xe3\x90u\x91\x07\x12\xec;$-\xda\x93\xf0o\xe1\xdf.\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0']
Bad pipe message: %s [b"/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x00"]
Bad pipe message: %s [b'8\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00']
Bad pipe message: %s [b'\x00\t127.0.0.1']
Bad pipe message: %s [b'9\xcdZ\x94\x84\xb9\x7f\x0c\xbe\x00\xed\x00\x8f\x00\x14\xbf\xc8\xf3\x00\x00\xa2', b'\xc0\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00:\x00\x89\xc0\x0f\xc0\x05\x005\x00\x84\xc0\x13\xc0\t\x003\x002\x001\x000\x00\x9a\x0

Selecting people from each county in year=2026...14 mins per year
Selecting people from each county in year=2027...14 mins per year
Selecting people from each county in year=2028...14 mins per year
Selecting people from each county in year=2029...14 mins per year


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 38.0min


Selecting people from each county in year=2030...14 mins per year
Selecting people from each county in year=2031...14 mins per year
Selecting people from each county in year=2032...14 mins per year
Selecting people from each county in year=2033...14 mins per year


In [7]:
# check outputs
df_2035 = pd.read_parquet(os.path.join(datadir, 'county_samples/county_sample_2035.parquet'))

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,destination_county,engine,segment,efficiency,year
10297,1696643382025952548,10234240303577394052,PRIVATE_AUTO,WORK,HOME,0 days 07:20:40,0 days 07:25:17,1.883100,nan,530019505001,46.811524,-119.151463,530019505002,46.811462,-119.182942,residential,single_family,residential,single_family,industrial,industrial,industrial,industrial,46.823401,-119.155263,46.815662,-119.174829,thursday,12555405632766289807,530019505001,530019505002,nan,53001950500,53001950500,nan,35_64,54.0,F,white,hispanic_or_latino,20000_40000,25931.0,employed,some_college,not_attending_school,naics423710,spouse,0,nan,driving,owner,same_house,2_person,40000_75000,55103.0,married_couple,2,single_family,core,spanish,46.823401,-119.155263,46.815662,-119.174829,NaN,NaN,worked_in_person,WORK,"Adams County, WA",EV,Commercial Crossover,0.3,2035
10298,9032998044962274151,10234240303577394052,PRIVATE_AUTO,SHOP,WORK,0 days 08:14:41,0 days 08:22:48,1.508437,nan,530019505002,46.811462,-119.182942,530019504001,46.828923,-119.159610,industrial,industrial,industrial,industrial,commercial,retail,commercial,retail,46.815662,-119.174829,46.826590,-119.158414,thursday,12555405632766289807,530019505001,530019505002,nan,53001950500,53001950500,nan,35_64,54.0,F,white,hispanic_or_latino,20000_40000,25931.0,employed,some_college,not_attending_school,naics423710,spouse,0,nan,driving,owner,same_house,2_person,40000_75000,55103.0,married_couple,2,single_family,core,spanish,46.823401,-119.155263,46.815662,-119.174829,NaN,NaN,worked_in_person,PUBLIC,"Adams County, WA",EV,Commercial Crossover,0.3,2035
10299,15263364619448657250,10234240303577394052,PRIVATE_AUTO,WORK,SHOP,0 days 09:12:34,0 days 09:17:44,1.508437,nan,530019504001,46.828923,-119.159610,530019505002,46.811462,-119.182942,commercial,retail,commercial,retail,industrial,industrial,industrial,industrial,46.826590,-119.158414,46.815662,-119.174829,thursday,12555405632766289807,530019505001,530019505002,nan,53001950500,53001950500,nan,35_64,54.0,F,white,hispanic_or_latino,20000_40000,25931.0,employed,some_college,not_attending_school,naics423710,spouse,0,nan,driving,owner,same_house,2_person,40000_75000,55103.0,married_couple,2,single_family,core,spanish,46.823401,-119.155263,46.815662,-119.174829,NaN,NaN,worked_in_person,WORK,"Adams County, WA",EV,Commercial Crossover,0.3,2035
10300,8036733709150494378,10234240303577394052,PRIVATE_AUTO,WORK,HOME,0 days 09:46:21,0 days 09:50:57,1.883100,nan,530019505001,46.811524,-119.151463,530019505002,46.811462,-119.182942,residential,single_family,residential,single_family,industrial,industrial,industrial,industrial,46.823401,-119.155263,46.815662,-119.174829,saturday,12555405632766289807,530019505001,530019505002,nan,53001950500,53001950500,nan,35_64,54.0,F,white,hispanic_or_latino,20000_40000,25931.0,employed,some_college,not_attending_school,naics423710,spouse,0,nan,driving,owner,same_house,2_person,40000_75000,55103.0,married_couple,2,single_family,core,spanish,46.823401,-119.155263,46.815662,-119.174829,NaN,NaN,worked_in_person,WORK,"Adams County, WA",EV,Commerc

In [3]:
df_2034 = pd.read_parquet(os.path.join(datadir, 'county_samples/county_sample_2034.parquet'))

In [6]:
df_2034[['person_id', 'distance_miles', 'destination_county', 'building_type', 'engine', 'segment', 'efficiency', 'year']]

,person_id,distance_miles,destination_county,building_type,engine,segment,efficiency,year
10297,10234240303577394052,1.883100,"Adams County, WA",single_family,EV,Commercial Crossover,0.3,2034
10298,10234240303577394052,1.508437,"Adams County, WA",single_family,EV,Commercial Crossover,0.3,2034
10299,10234240303577394052,1.508437,"Adams County, WA",single_family,EV,Commercial Crossover,0.3,2034
10300,10234240303577394052,1.883100,"Adams County, WA",single_family,EV,Commercial Crossover,0.3,2034
10301,10234240303577394052,1.611954,"Adams County, WA",single_family,EV,Commercial Crossover,0.3,2034
...,...,...,...,...,...,...,...,...
51722247,12974764336925645660,2.129131,"Yakima County, WA",single_family,PHEV,Personal Truck/SUV,0.9,2034
51722248,12974764336925645660,2.131551,"Yakima County, WA",single_family,PHEV,Personal Truck/SUV,0.9,2034
51722249,12974764336925645660,2.129131,"Yakima County, WA",single_family,PHEV,Personal Truck/SUV,0.9,2034
51725545,12975743187318417717,2.232699,"Yakima County, WA",single_family,PHEV,Personal Truck/SUV,0.9,2034


# why are they all the same length?

In [18]:
year_list = np.arange(2022, 2024, 1)
df_list = []
for year in year_list:
    print(year)
    df_i = pd.read_parquet(os.path.join(datadir, 'county_samples/county_sample_{}.parquet'.format(year)))
    print(len(df_i.drop_duplicates(subset=['person_id'])))
    # df_list.append()

# df_out = pd.concat(df_list)

2022
252329
2023
252329


In [18]:
len(pop_df), len(df)

(5046603, 25405990)

# make toy data

In [22]:
person_ids = [1, 2, 2, 3, 3, 4, 4, 5, 6, 6]
counties = ['King', 'Adams', 'Adams', 'King', 'King', 'Adams', 'Adams', 'Adams', 'King', 'King']

trips = pd.DataFrame({'person_id': person_ids, 'county': counties})
pop = trips.drop_duplicates(subset=['person_id'])[['person_id', 'county']]
cnty_list = 

In [24]:
pop.groupby('county').count()

,person_id
county,
Adams,3
King,3


In [14]:
ev_cond = stock_rollover['Powertrain'].isin(['EV', 'PHEV']).reset_index(drop=True)
# ev_cond = (stock_rollover['Powertrain']=='EV') | (stock_rollover['Powertrain']=='PHEV')
# nev_df = stock_rollover[(stock_rollover['Powertrain']=='EV')].copy()
nev_df = stock_rollover[ev_cond].copy()
nev_df = nev_df[nev_df['domicile'] != 'other'].copy()
nev_df.drop(columns=['Unnamed: 0'], inplace=True)

ev_sfh_only = nev_df[nev_df['domicile']=='sfh'].copy()
ev_sfh_only = ev_sfh_only[ev_sfh_only['Powertrain']=='EV'].copy()
ev_sfh_only = ev_sfh_only[ev_sfh_only['Vehicle_type']=='Personal Sedan'].copy()
ev_sfh_only

,County,Vehicle_type,domicile,Powertrain,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
36,Adams,Personal Sedan,sfh,EV,12,357,724,984,1223,1389,1526,1608,1709,1766,1806,1859,1900,1924
84,Asotin,Personal Sedan,sfh,EV,18,266,529,717,892,1013,1116,1182,1258,1300,1335,1375,1408,1433
132,Benton,Personal Sedan,sfh,EV,458,1934,4076,5788,7520,8811,10016,10805,11852,12551,13153,13956,14645,15203
180,Chelan,Personal Sedan,sfh,EV,238,950,1728,2303,2836,3208,3528,3714,3953,4076,4170,4291,4389,4465
228,Clallam,Personal Sedan,sfh,EV,288,1151,2060,2727,3340,3767,4132,4338,4610,4756,4864,5017,5141,5240
276,Clark,Personal Sedan,sfh,EV,2253,5095,9414,12884,16463,19123,21638,23277,25505,27005,28334,30178,31785,33135
316,Columbia,Personal Sedan,sfh,EV,6,59,112,149,183,205,224,234,248,255,261,264,267,274
364,Cowlitz,Personal Sedan,sfh,EV,198,1175,2401,3341,4244,4889,5458,5806,6270,6548,6786,7109,7386,7622
412,Douglas,Personal Sedan,sfh,EV,83,549,1065,1447,1802,2051,2266,2391,2548,2634,2697,2786,2855,2905
452,Ferry,Personal Sedan,sfh,EV,15,100,175,226,271,302,326,339,354,360,366,368,371,369


In [17]:
reduced_df = []
unique_df = df.drop_duplicates(subset=['person_id'])[['person_id', 'destination_county', 'building_type']]
for year in [2023, 2025, 2030, 2035]:
    
    num_to_select = ev_sfh_only[str(year)].sum()
    selected = unique_df.person_id.sample(n=num_to_select, replace=False, random_state=42)
    
    # grab only those selected people from the original dataframe
    year_df = df[(df['person_id'].isin(selected))].copy()
    year_df['year'] = year
    reduced_df.append(year_df)

    print(ev_sfh_only[str(year)].sum(), len(year_df.drop_duplicates(subset=['person_id'])))
    
final_df = pd.concat(reduced_df)

84189 84189
202943 202943
380651 380651
478275 478275


In [11]:
final_df[final_df['year'] == 2023]

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,destination_county,year
76,18316497155116191323,10000027284534762868,PRIVATE_AUTO,SHOP,WORK,0 days 19:20:08,0 days 19:46:17,24.728563,nan,530579515003,48.523396,-122.218307,530730012013,48.723092,-122.479946,mixed_use,industrial,industrial,industrial,commercial,retail,commercial,retail,48.511020,-122.228264,48.733093,-122.470342,saturday,6807149718198083678,530730012013,530579515003,nan,53073001201,53057951500,nan,35_64,41.0,M,white,not_hispanic_or_latino,40000_80000,50507.0,employed,some_college,undergraduate,naics31_33,child,0,nan,driving,renter,within_us,5_person,125000_plus,146471.0,married_couple,3_plus,single_family,core,english,48.715497,-122.485185,48.511020,-122.228264,NaN,NaN,worked_in_person,PUBLIC,"Whatcom County, WA",2023
77,5744120661337032474,10000027284534762868,PRIVATE_AUTO,HOME,SHOP,0 days 20:01:07,0 days 20:09:20,2.048920,nan,530730012013,48.723092,-122.479946,530730012013,48.723092,-122.479946,commercial,retail,commercial,retail,residential,single_family,residential,single_family,48.733093,-122.470342,48.715497,-122.485185,saturday,6807149718198083678,530730012013,530579515003,nan,53073001201,53057951500,nan,35_64,41.0,M,white,not_hispanic_or_latino,40000_80000,50507.0,employed,some_college,undergraduate,naics31_33,child,0,nan,driving,renter,within_us,5_person,125000_plus,146471.0,married_couple,3_plus,single_family,core,english,48.715497,-122.485185,48.511020,-122.228264,NaN,NaN,worked_in_person,HOME,"Whatcom County, WA",2023
79,9725345086549260082,10000027284534762868,PRIVATE_AUTO,HOME,MAINTENANCE,0 days 20:36:27,0 days 20:40:56,0.226769,nan,530730012023,48.708440,-122.481150,530730012013,48.723092,-122.479946,commercial,office,commercial,office,residential,single_family,residential,single_family,48.714490,-122.486678,48.715497,-122.485185,thursday,6807149718198083678,530730012013,530579515003,nan,53073001201,53057951500,nan,35_64,41.0,M,white,not_hispanic_or_latino,40000_80000,50507.0,employed,some_college,undergraduate,naics31_33,child,0,nan,driving,renter,within_us,5_person,125000_plus,146471.0,married_couple,3_plus,single_family,core,english,48.715497,-122.485185,48.511020,-122.228264,NaN,NaN,worked_in_person,HOME,"Whatcom County, WA",2023
104,1496420818013793582,10000039559268314038,PRIVATE_AUTO,HOME,SOCIAL,0 days 14:52:35,0 days 15:02:36,2.065542,nan,530530726022,47.316534,-122.758200,530530726022,47.316534,-122.758200,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.300459,-122.763112,47.305191,-122.775970,saturday,9203986852395706118,530530726022,nan,nan,53053072602,nan,nan,65_plus,73.0,M,white,not_hispanic_or_latino,40000_80000,56145.0,notinlf,some_college,not_attending_school,not_working,head_of_household,0,nan,not_working,owner,same_house,2_person,40000_75000,66880.0,married_couple,1,single_family,core,english,47.305191,-122.775970,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME,"Pierce County, WA",2023
150,3406319916074195357,10000052672857184456,PRIVATE_AUTO,RECREATION,HOME,0 days 09:10:18,0 days 09:22:05,9.115

In [19]:
ev_sfh_only['County'].nunique()

39

In [28]:
nev_df.groupby(['County', 'Vehicle_type', 'domicile', 'Powertrain', '2022']).sum()

2023   2024   2025   
County Vehicle_type         domicile Powertrain 2022                       
Adams  Commercial Crossover mfh      EV         0        0      0      0  \
                                     PHEV       0        0      0      0   
                            sfh      EV         0        0      1      1   
                                     PHEV       0        1      1      1   
       Commercial Sedan     mfh      EV         0        0      0      0   
...                                                    ...    ...    ...   
Yakima Personal Sedan       sfh      PHEV       98    2433   4815   6453   
       Personal Truck/SUV   mfh      EV         34    1780   3504   5143   
                                     PHEV       18    1063   1757   2265   
                            sfh      EV         133   6521  12813  18796   
                                     PHEV       65    3862   6381   8226   

                                                       2026   2027   2028   
County Vehicle_type         domicile Powertrain 2022                        
Adams  Commercial Crossover mfh      EV         0         0      0      0  \
                                     PHEV       0         0      0      0   
                            sfh      EV         0         1      2      2   
                                     PHEV       0         1      1      1   
       Commercial Sedan     mfh      EV         0         0      0      0   
...                                                     ...    ...    ...   
Yakima Personal Sedan       sfh      PHEV       98     7994   9117  10046   
       Personal Truck/SUV   mfh      EV         34     6768   8393   9963   
                                     PHEV       18     2617   2945   3170   
                            sfh      EV         133   24713  30637  36335   
                                     PHEV       65     9505  10701  11522   

                                                       2029   2030   2031   
County Vehicle_type         domicile Powertrain 2022                        
Adams  Commercial Crossover mfh      EV         0         1      1      1  \
                                     PHEV       0         0      0      0   
                            sfh      EV         0         2      3      3   
                                     PHEV       0         1      1      1   
       Commercial Sedan     mfh      EV         0         0      0      0   
...                                                     ...    ...    ...   
Yakima Personal Sedan       sfh      PHEV       98    10703  11606  12252   
       Personal Truck/SUV   mfh      EV         34    11285  12851  14524   
                                     PHEV       18     3367   3593   3845   
                            sfh      EV         133   41141  46932  53211   
                                     PHEV       65    12233  13039  13915   

                                                       2032   2033   2034   
County Vehicle_type         domicile Powertrain 2022                        
Adams  Commercial Crossover mfh      EV         0         1      1      1  \
                                     PHEV       0         0      0      0   
                            sfh      EV         0         3      5      5   
                                     PHEV       0         1      1      1   
       Commercial Sedan     mfh      EV         0         0      0      0   
...                                                     ...    ...    ...   
Yakima Personal Sedan       sfh      PHEV       98    12842  13552  14206   
       Personal Truck/SUV   mfh      EV         34    16279  18064  19987   
                                     PHEV       18     4102   4380   4667   
                            sfh      EV         133   59811  66703  74132   
                                     PHEV       65    14799  15789  16813   

                                                       2035